Task 2: Application of linear models (both univariate and multivariate), ARIMA, and Prophet for price
forecasting and relationship analysis.

In [1]:
import pandas as pd
import numpy as np
import os

In [14]:
# Load in datasets
df_btc = pd.read_csv('../data/btc_data.csv')
df_eth = pd.read_csv('../data/eth_data.csv')
df_sp500 = pd.read_csv('../data/sp500_data.csv')
df_vix = pd.read_csv('../data/vix_data.csv')
df_gold = pd.read_csv('../data/gold_data.csv')

In [15]:
df_btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3758 entries, 0 to 3757
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3758 non-null   object 
 1   Close   3758 non-null   float64
 2   High    3758 non-null   float64
 3   Low     3758 non-null   float64
 4   Open    3758 non-null   float64
 5   Volume  3758 non-null   int64  
 6   Return  3758 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 205.6+ KB


In [16]:
df_btc

,Date,Close,High,Low,Open,Volume,Return
0,2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200,-0.071926
1,2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700,-0.069843
2,2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600,0.035735
3,2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100,-0.024659
4,2014-09-22,402.152008,406.915985,397.130005,399.100006,24127600,0.008352
...,...,...,...,...,...,...,...
3753,2024-12-27,94164.859375,97294.843750,93310.742188,95704.976562,52419934565,-0.017022
3754,2024-12-28,95163.929688,95525.898438,94014.289062,94160.187500,24107436185,0.010610
3755,2024-12-29,93530.226562,95174.875000,92881.789062,95174.054688,29635885267,-0.017167
3756,2024-12-30,92643.210938,94903.320312,91317.132812,93527.195312,56188003691,-0.009484
